In [49]:
import os
import json

import pandas as pd
from tqdm import tqdm
from lavis.common.utils import get_abs_path, get_cache_path

In [50]:
cc3m = pd.read_csv("downloaded_cc3m_report.tsv.gz", compression="gzip", sep="\t", names=["caption", "path", "dataset", "mimetype", "size", "status", "url"])

In [51]:
cc3m.iloc[15]

caption     functions of government : 1 . form a more perf...
path        /home/rafa/LAVIS-main/lavis/../cache/conceptua...
dataset                                                  cc3m
mimetype                                           image/jpeg
size                                                  31561.0
status                                                    200
url         http://images.slideplayer.com/2/765769/slides/...
Name: 15, dtype: object

In [52]:
len(cc3m)

3318333

In [61]:
cnt = 0

valid_records = []

for i, path in tqdm(enumerate(cc3m.path.unique()), total=len(cc3m.path.unique())):
    path = str(path)
    if os.path.exists(path):
        record = cc3m.iloc[i]
        valid_records.append({"image": record["path"], "caption": record["caption"]})

        cnt += 1

    else:
        if '/home/rafa/LAVIS-main/lavis/datasets/download_scripts/DownloadConceptualCaptions/cache/conceptual_caption/images' in path:
            # print(" {} ".format(path))
            path=os.path.join('/home/rafa/LAVIS-main/lavis/../cache/conceptual_caption/images/',path.lstrip('/home/rafa/LAVIS-main/lavis/datasets/download_scripts/DownloadConceptualCaptions/cache/conceptual_caption/images'))
            # print("fuck {} ".format(path))
            if os.path.exists(path):
                record = cc3m.iloc[i]
                valid_records.append({"image": path, "caption": record["caption"]})
                cnt += 1
                print("fuck {} ".format(path))


print("Found {} valid records".format(cnt))

100%|██████████| 2033390/2033390 [01:45<00:00, 19363.59it/s]

Found 1746981 valid records


In [62]:
len(valid_records)

1746981

In [63]:
valid_records[1]

{'image': '/home/rafa/LAVIS-main/lavis/../cache/conceptual_caption/images/2_2859920462.jpg',
 'caption': 'young confused girl standing in front of a wardrobe'}

In [64]:
from omegaconf import OmegaConf


config_path = get_abs_path("configs/datasets/conceptual_caption/defaults_3m.yaml")

ann_path = OmegaConf.load(
    config_path
).datasets.conceptual_caption_3m.build_info.annotations.train.storage[0]

ann_path = get_cache_path(ann_path)

if os.path.exists(ann_path):
    # abort
    print("{} already exists".format(ann_path))
else:
    # Save the valid records to a json file
    with open(ann_path, "w") as f:
        f.write(json.dumps(valid_records))